# Aprendizaje Automático para Datos en Grafos - Laboratorio 1
## Introducción al procesamiento de grafos, NetworkX y PyTorch Geometric

En este laboratorio tomaremos un dataset real, generaremos grafos a partir de él y lo analizaremos mediante la biblioteca **[NetworkX](https://networkx.org/)**. En el camino, aprovechamos para presentar la biblioteca **pandas**, excelente para levantar datos y poder procesarlos de manera rápida y cómoda. También empezaremos a familarizarnos con **PyTorch Geometric**, una biblioteca que nos será útil para trabajar con redes neuronales en grafos (GNNs).

Como excusa para esto, usaremos el dataset de Enron. Los detalles del escándalo se pueden consultar en https://en.wikipedia.org/wiki/Enron_scandal. Durante la investigación se publicaron los mails que recibieron y enviaron varios empleados de Enron entre noviembre de 1998 y junio de 2002. El dataset completo se puede consultar en http://www.cs.cmu.edu/~enron/. Aquí usaremos una versión curada y resumida (por ejemplo, no contiene el cuerpo de los mensajes) que se puede consultar en http://cis.jhu.edu/~parky/Enron/enron.html.

El objetivo es que aquellos que nunca tuvieron contacto con alguna de estas bibliotecas aprendan mínimamente a usarlas. Para el curso, se pide que entreguen las respuestas a las preguntas planteadas, pero cualquier exploración por fuera del camino trazado es más que bienvenida.

## Generando el grafo

In [ ]:
# importemos bibliotecas que vamos a usar
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# bajemos el dataset (ver http://cis.jhu.edu/~parky/Enron/enron.html por los detalles)
!wget http://cis.jhu.edu/~parky/Enron/employees
!wget http://cis.jhu.edu/~parky/Enron/execs.email.linesnum

In [ ]:
# carguemos los datos
df_mails = pd.read_csv('execs.email.linesnum', names=['time','from','to'], sep=' ')
df_employees = pd.read_csv('employees', sep='\t', names=['mail', 'name and more'])


En la variable `df_mails` tenemos un [`DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) de pandas con el id de quien (columna `from`) le envió un mail a quien (`to`) y cuándo (`time`). A su vez, el usuario de mail y otros datos de cada usuario está en el dataframe `df_employees`. Un dataframe se puede pensar como una tabla indexada, pero además pandas da muchas facilidades, algunas de las cuales vamos a aprovechar para procesar los datos y generar el grafo.

In [ ]:
# calculamos las fechas a partir del timestamp (de segundos desde el 1/1/1970 a una fecha)
df_mails['date'] = pd.to_datetime(df_mails.time, unit='s')

# no sé porque hay fechas del 79. Olimpicamente las borramos.
df_mails = df_mails[df_mails.date.dt.year>1980]

df_mails.head()

### Generación del grafo de todo el período

Primero vamos a crear un grafo que abarque todos los mails.


In [ ]:
# cuento cuántos mails hubo entre dos usuarios
mails_intercambiados = df_mails.groupby(['from', 'to']).count().reset_index()
mails_intercambiados.head()


In [ ]:
# las columnas "time" y "date" tienen todo lo mismo, asi que arbitrariamente cambio una a "weight" para usarlo como peso en el grafo
mails_intercambiados.rename(columns={'time':'weight'}, inplace=True)
mails_intercambiados.head()

In [ ]:
# Y acá viene lo interesante: pandas se integra con networkx.
G = nx.from_pandas_edgelist(mails_intercambiados, source='from', target='to', edge_attr='weight', create_using=nx.DiGraph)

# Saco self loops
G.remove_edges_from(nx.selfloop_edges(G))

# dibujamos el grafo sin hacer mucho esfuerzo ...
nx.draw_networkx(G)
plt.show()

In [ ]:
# ... pero no se aprecia mucho, porque los nodos quedan muy apretados.

# Así que ahora nos vamos a poner un poco más las pilas
posiciones = nx.circular_layout(G)
edges = G.edges()
weights = np.array([G[u][v]['weight'] for u,v in edges])

between_dict = nx.betweenness_centrality(G)
between = np.array(list(between_dict.values()))

plt.figure(figsize=(12,12))
nx.draw_networkx_nodes(G, pos=posiciones, node_color=10*np.log(1+between/(np.min(between)+1e-9)), cmap='Blues')
nx.draw_networkx_edges(G, alpha=0.1, width=np.log10(weights+1), pos=posiciones)
nx.draw_networkx_labels(G, pos=posiciones, font_color='black')
plt.title('Grafo de mails intercambiados en todo el período.\n Ancho de la arista proporcional a la cantidad de mails (en escala logarítmica).\n \
  Intensidad del color del nodo proporcional a su betweeness (en escala logarítmica).', fontsize=18)
plt.show()

### Interfaz de NetworkX con NumPy

In [ ]:
# Además de tener interfaz hacia pandas, NetworkX puede trabajar con NumPy y matrices

# por ejemplo, la matriz de adyacencia se obtiene así de sencillo
G_np = nx.to_numpy_array(G,nodelist=range(G.number_of_nodes()))
# y la ploteo usando seaborn
sns.heatmap(np.log10(G_np+1), cmap='Greys')
plt.show()
# o si quisiera mirar solo la conectividad...
sns.heatmap(G_np>0, cmap='Greys')
plt.show()


## Análisis del grafo completo

Ahora deberá usar la API de Networkx o NumPy para calcular distintos indicadores del grafo `G`:


1.   Número de aristas dirigidas (arcos).
2.   Número de aristas no-dirigidas (es decir, si existe al menos una de las dos direcciones entre un par de nodos (i,j), entonces se cuenta como 1).
3.   Número de arcos mutuos (es decir, pares de nodos con "ida y vuelta").
4.   Número de nodos con in-degree igual a 0 (o sea, empleados que no recibieron mails). ¿A qué personas corresponde?
5.   Número de nodos con out-degree igual a 0 (o sea, empleados que no enviaron mails). ¿A qué personas corresponde?
6.   El número de empleados que fueron contactados por más de 30 empleados. Re-haga el dibujo del grafo marcando estos nodos en rojo y haciendo aparecer su nombre.
7.   El número de empleados que contactaron a más de 30 empleados. Re-haga el dibujo del grafo marcando estos nodos en rojo y haciendo aparecer su nombre.
8.   El histograma del grado de los nodos (in y out). Se puede utilizar por ejemplo la herramienta histplot de seaborn.



## Análisis temporal del grafo

Hasta ahora miramos la totalidad de los datos e ignoramos la componente temporal del mismo. Ahora vamos a analizar mínimamente cómo cambia el grafo con el tiempo.

In [ ]:
# los voy a agrupar por semana, asi que primero me fijo a que semana corresponde cada mail y lo agrego a df_mails
df_mails['week'] = df_mails.date.dt.to_period('W')
print(df_mails.head())

# Agrupo por semana. Esto genera un objeto GroupBy sobre el que puedo iterar, y contiene
# los datos correspondientes a cada semana
grouped_week = df_mails.groupby('week')
# lista que contendrá los grafos correspondientes a cada semana
grafos = []
# lista que contendrá las semanas propiamente dichas. Se puede usar para identificar tiempo más adelante.
semanas = []

for semana, grupo_mails in grouped_week:
    # básicamente repito lo que hice antes para el grafo grande, pero ahora por semana.
    # Voy a ir guardando los grafos en una lista. Quizá no sea lo más eficiente, pero el
    # dataset no es muy grande.

    # cuento cuantos mails hubo en esta semana entre dos usuarios
    mails_intercambiados = grupo_mails.groupby(['from', 'to']).count().reset_index()
    # las columnas tienen todo lo mismo, asi que arbitrariamente cambio una a "weight" para usarlo como peso en el grafo
    mails_intercambiados.rename(columns={'week':'weight'}, inplace=True)
    G = nx.from_pandas_edgelist(mails_intercambiados, source='from', target='to', edge_attr='weight', create_using=nx.DiGraph)

    # Saco self loops
    G.remove_edges_from(nx.selfloop_edges(G))

    # agrego el nuevo grafo a la lista
    grafos.append(G)
    semanas.append(semana)



In [ ]:
# vamos a mirar algunos indicadores

numero_de_nodos = [grafo.number_of_nodes() for grafo in grafos]
numero_de_arcos = [grafo.number_of_edges() for grafo in grafos]
pd.DataFrame({'n_nodos':numero_de_nodos, 'n_arcos':numero_de_arcos}, index=semanas).plot(figsize=(12,6))
plt.grid()
plt.legend(['número de nodos', 'número de arcos'])
plt.show()


### Cambios en el grafo:
9. Elija dos indicadores de nodo e indique cuál fue la persona más importante para cada instante de tiempo según cada indicador. Justifique su elección del indicador.  Compárelo con el grafo "completo" (i.e. considerando todo el tiempo).
10. Considere algunos indicadores a nivel de grafo y trate de identificar algún evento en el escándalo (la figura 8 de https://arxiv.org/abs/1403.0989 tiene un timeline muy prolijo que puede ayudar). Con toda seguridad el lanzamiento de Enron online y la asunción de Stephen Cooper como CEO son fácilmente detectables.

## Una introducción a Pytorch Geometric (PyG)
**[PyTorch Geometric](https://github.com/rusty1s/pytorch_geometric)** es una bilbioteca de Python para trabajar con aprendizaje profundo en grafos, a través de las redes neuronales de grafos (Graph Neural Networks o GNNs). La biblioteca es una extensión de **[PyTorch](https://pytorch.org/)**, un framework de deep learning muy utilizado.

In [ ]:
# Instalo PyG
!pip install torch-geometric

PyG tiene varios datasets de grafos en el paquete **[torch_geometric.datasets](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets)**. En esta parte trabajaremos con un dataset bastante utilizado para hacer pruebas de concepto en el área: el [**Zachary's karate club network**](https://en.wikipedia.org/wiki/Zachary%27s_karate_club).

In [ ]:
from torch_geometric.datasets import KarateClub

dataset = KarateClub()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Cantidad de grafos: {len(dataset)}')
print(f'Cantidad de features: {dataset.num_features}')
print(f'Cantidad de clases: {dataset.num_classes}')

El dataset está formado por un único grafo, cada nodo del grafo tiene asociado un vector en $\mathbb{R}^{34}$ (llamado vector de *features*), y hay 4 clases de nodos. Veamos en más detalle algunas estadísticas del grafo:

In [ ]:
# Miro el primer grafo
data = dataset[0]

print(data)
print('==============================================================')

# Estadísticas del grafo.
print(f'Cantidad de nodos: {data.num_nodes}')
print(f'Cantidad de aristas: {data.num_edges}')
print(f'Grado promedio de nodo: {(2*data.num_edges) / data.num_nodes:.2f}')
print(f'Tiene nodos aislados: {data.has_isolated_nodes()}')
print(f'Tiene self-loops: {data.has_self_loops()}')
print(f'Es no dirigido: {data.is_undirected()}')

Un grafo en PyG está representado por un objeto de tipo [`Data`](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.Data). Cada uno de estos objetos tiene al menos 5 atributos:
- **`x`**: es la matriz de features del grafo (es decir, la matriz cuyas columnas son los featues de cada nodo del grafo). Es un objeto de tipo [`tensor`](https://pytorch.org/docs/stable/tensors.html), el tipo nativo de torch para guardar matrices (el equivalente a [`ndarray`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) en numpy).
- **`edge_index`**: es la matriz de conectividad del grafo, en formato [COO](https://en.wikipedia.org/wiki/Sparse_matrix#Coordinate_list_(COO)). Este formato es muy útil para guardar matrices *ralas* o *esparsas* (matrices con muchos ceros en sus entradas). Consiste en guardar solamente la lista de nodos que están conectados, en lugar de guardar toda la matriz de adyacencia del grafo.
- **`y`**: es la matriz de etiquetas de cada nodo (en este ejemplo, la matriz que nos dice a cuál de las 4 clases pertenece cada nodo).
- **`train_mask`**: matriz binaria que indica qué nodos de nuestro dataset son parte del conjunto de entrenamiento. Será útil más adelante cuando queramos por ejemplo estimar a qué clase pertenece cada nodo.
- **`edge_attr`**: matriz de features de cada arista del grafo. Como el grafo con el que estamos trabajando es simple, en este caso las aristas no tienen features, pero en el caso que tengamos un grafo dirigido, en `edge_attr` podríamos guardar los pesos de cada arista.

In [ ]:
print('data.x')
print('========================================')
print(data.x)
print('\ndata.edge_index')
print('=========================================')
print(data.edge_index.t())
print('\ndata.y')
print('=========================================')
print(data.y)
print('\ndata.train_mask')
print('=========================================')
print(data.train_mask)
print('\ndata.edge_attr')
print('=========================================')
print(data.edge_attr)

PyG nos proporciona una interfaz simple para convertir un grafo a formato de Networkx

In [ ]:
from torch_geometric.utils import to_networkx
G = to_networkx(data, to_undirected=True)
nx.draw_networkx(G,node_color=data.y,pos=nx.spring_layout(G, seed=42))

## Verificando empíricamente las propiedades de la matriz laplaciana
La idea de los siguientes ejercicios es verificar empíricamente las propiedades de la matriz laplaciana que se demuestran en el ejercicio opcional de abajo.

11. Calcular la matriz laplaciana $\mathbf{L}$ del grafo del karate club. Se sugiere utilizar alguna función del subpaquete [`torch_geometric.utils`](https://pytorch-geometric.readthedocs.io/en/latest/modules/utils.html).
12. Vertificar que $\mathbf{L}$ tiene valor propio 0, y que el vector de unos $[1,1,\dots,1]^\top$ es vector propio con valor propio 0. El subpaquete [`torch.linalg`](https://pytorch.org/docs/stable/linalg.html) puede ser de utilidad.
13. Verificar que $\mathbf{L}$ es semidefinida positiva.
14. Armar una matriz $\tilde{\mathbf{B}}$ como la de la parte 2. del ejercicio opcional y verificar que $\mathbf{L}=\tilde{\mathbf{B}}\tilde{\mathbf{B}}^\top$. Se sugiere utilizar la función [`networkx.incidence_matrix`](https://networkx.org/documentation/stable/reference/generated/networkx.linalg.graphmatrix.incidence_matrix.html).

# Ejercicio opcional: demostrar algunas propiedades de la matriz laplaciana

Sea $G(V,E)$ un grafo simple (no dirigido, sin pesos y sin auto-aristas), con cantidad de vértices $N_v = |V|$ y cantidad de aristas $N_e = |E|$, y matriz de adyacencia $\mathbf{A}$. Sea $\mathbf{D} = \text{diag}(d_{1},\dots, d_{N_v})$ la matriz de grados y $\mathbf{L}=\mathbf{D}- \mathbf{A}$ la matriz laplaciana de $G$.

1. Verificar que el vector de unos $[1,1,\dots,1]^\top$ es vector propio de $\mathbf{L}$ con valor propio 0.
2. Aunque $G$ es un grafo no dirigido, podemos asignarle a cada arista en $E$ una orientación "virtual" arbitraria: para cada arista elegimos arbitrariamente uno de sus vértices como nodo inicial y el otro como nodo final. Dada esa asignación, definimos la *matriz de incidencia signada* $\tilde{\mathbf{B}} \in \{-1,0,1\}^{N_v\times N_e}$ con entrada $i,e$ dada por:
$$\tilde{\mathbf{B}}_{ie} = \left\lbrace
\begin{array}{r l}
1,& \text{si el vértice } i \text{ es el nodo inicial de la arista } e\\
-1,& \text{si el vértice } i \text{ es el nodo final de la arista } e\\
0,& \text{en otro caso}
\end{array}
\right. .
$$
Demostrar que la matriz laplaciana puede descomponerse como $\mathbf{L}=\tilde{\mathbf{B}}\tilde{\mathbf{B}}^\top$.

3. Sea $\mathbf{x} = [x_1,\dots,x_{N_v}]^\top \in \mathbb{R}^{N_v}$ un vector cualquiera. Usando el resultado de la parte 2, mostrar que
$$\mathbf{x}^\top \mathbf{L}\mathbf{x} = \sum_{(i,j) \in E} (x_i-x_j)^2$$
Concluir que $\mathbf{L}$ es una matriz simétrica y semidefinida positiva.

4. Demostrar que si $G$ no es conexo entonces $\mathbf{L}$ es diagonal por bloques, donde cada bloque corresponde a la matriz laplaciana de cada una de las componentes conexas de $G$. Probar que en este caso el subespacio propio asociado al valor propio 0 tiene dimensión mayor o igual a 2.

# Reconocimientos

La primera parte de la sección '**Una introducción a PyTorch Geometric**' está basada en [este notebook](https://colab.research.google.com/drive/16tqEHKOLUgYvXKx1V3blfYGpQb1_09MG?usp=sharing#scrollTo=bbny-iTO7NQN) del curso **[CS224W: Machine Learning with Graphs](http://web.stanford.edu/class/cs224w)** de Stanford.